# **Trabajo Práctico Final**

Proyecto de Machine Learning para Logística Aérea: Optimización de tiempos de transporte aéreo

Descripción General

Comprensión del Negocio

**IMPORTACIÓN DE LIBRERÍAS**

In [4]:
# Instalar CatBoost, ya que no viene preinstalada en Google Colab
%pip install catboost

# Librerías para manipulación y análisis de datos
import pandas as pd
import numpy as np

# Librerías para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de Scikit-learn para el preprocesamiento y modelado
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Librerías para modelos de regresión
# Un modelo lineal simple como punto de partida (benchmark)
from sklearn.linear_model import LinearRegression

# Modelos de ensemble potentes para mayor precisión
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb # Se importa como 'xgb' por convención
import lightgbm as lgb # Se importa como 'lgb' por convención
from catboost import CatBoostRegressor

# Métricas de evaluación para regresión
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuración para mostrar gráficos en el notebook
%matplotlib inline

# Configuración para ignorar advertencias y mejorar la visibilidad
import warnings
warnings.filterwarnings('ignore')

# Configuración de Seaborn
sns.set()

# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

#Drive
import os
from google.colab import drive

**VINCULAR GOOGLE DRIVE**

In [14]:
# Montar Drive
drive.mount('/content/drive')

# Definir ruta del archivo kaggle.json en Drive
DRIVE_KAGGLE_PATH = 'Mi unidad/kaggle/kaggle.json'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# 3. Crear el directorio de configuración de Kaggle si no existe
KAGGLE_CONFIG_DIR = os.path.expanduser('~/.kaggle')
if not os.path.exists(KAGGLE_CONFIG_DIR):
    print(f"Creando directorio: {KAGGLE_CONFIG_DIR}")
    os.makedirs(KAGGLE_CONFIG_DIR)

# 4. Copiar el archivo desde Drive al directorio de Kaggle
if os.path.exists(DRIVE_KAGGLE_PATH):
    print(f"Copiando {DRIVE_KAGGLE_PATH} a {KAGGLE_CONFIG_DIR}/kaggle.json")
    !cp "{DRIVE_KAGGLE_PATH}" {KAGGLE_CONFIG_DIR}/kaggle.json

    # 5. Dar los permisos necesarios (obligatorio)
    !chmod 600 {KAGGLE_CONFIG_DIR}/kaggle.json
    print("\n¡Autenticación de Kaggle completada con éxito!")
else:
    print(f"\n¡ADVERTENCIA! No se encontró el archivo kaggle.json en: {DRIVE_KAGGLE_PATH}")
    print("Asegúrate de haberlo subido a tu Drive en esa ubicación.")


¡ADVERTENCIA! No se encontró el archivo kaggle.json en: Mi unidad/kaggle/kaggle.json
Asegúrate de haberlo subido a tu Drive en esa ubicación.


**IMPORTACIÓN DE DATASET**

In [3]:
# Instalar la librería de la API de Kaggle
%pip install kaggle

from google.colab import files
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi


OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [ ]:
print("Sube tu archivo kaggle.json (lo obtienes desde tu perfil de Kaggle):")
uploaded = files.upload()

In [ ]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_CONFIG_DIR'] = "/content"
# Create the default Kaggle config directory if it doesn't exist
os.makedirs("/root/.config/kaggle", exist_ok=True)

# Find the uploaded kaggle.json file, even if it was renamed by Colab
kaggle_json_filename = None
for filename in uploaded.keys():
    if 'kaggle.json' in filename:
        kaggle_json_filename = filename
        break

if kaggle_json_filename is None:
    print("Error: kaggle.json file not found in the uploaded files.")
else:
    with open("/content/.kaggle/kaggle.json", "w") as f:
        f.write(uploaded[kaggle_json_filename].decode("utf-8"))
    os.chmod("/content/.kaggle/kaggle.json", 0o600)
    # Copy kaggle.json to the default Kaggle config directory
    os.rename("/content/.kaggle/kaggle.json", "/root/.config/kaggle/kaggle.json")

In [ ]:
print("\nDescargando el dataset desde Kaggle...")
api = KaggleApi()
api.authenticate()
api.dataset_download_files('sofiayasmintschopp/datasetvuelosarg', path='./', unzip=True)

In [ ]:
# Cargar el archivo CSV en un DataFrame de pandas
# Asegúrate de que este es el nombre correcto del archivo CSV
df = pd.read_csv('DatasetVuelosArg.csv', sep=';')

**INSPECCIÓN Y ANÁLISIS DE DATOS INICIAL**

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

**LIMPIEZA DE DATOS**

In [ ]:
# ELIMINACIÓN DE COLUMNAS INNECESARIAS

columnas_a_mantener = [
    'Origin City', 'Origin Cntry',
    'Destin City', 'Destin Cntry', 'Service Level', 'Actual Weight',
    'Charge Weight', 'Pickup Date', 'Actual Departure', 'Actual Arrival',
    'Goods Description', 'Pieces', 'Estimated Arrival'
]

df = df[columnas_a_mantener]
print(df.columns)

In [ ]:
df.head()

In [ ]:
# MANEJO DE VALORES FALTANTES

# COMPROBACIÓN DE VALORES FALTANTES

print(df.isnull().sum())

In [ ]:
# ELIMINAR FILAS CON FECHAS DE SALIDA Y ARRIBO FALTANTES

df.dropna(subset=['Pickup Date', 'Actual Departure', 'Actual Arrival'], inplace=True)

**CONVERSIÓN DE DATOS**

In [ ]:
# CONVERSIÓN DE FECHAS A FORMATO DATETIME

df['Pickup Date'] = pd.to_datetime(df['Pickup Date'])
df['Actual Departure'] = pd.to_datetime(df['Actual Departure'])
df['Actual Arrival'] = pd.to_datetime(df['Actual Arrival'])
df['Estimated Arrival'] = pd.to_datetime(df['Estimated Arrival'])

**CREACIÓN DE VARIABLE OBJETIVO - TIEMPO DE TRÁNSITO**

In [ ]:
df['dias_transito'] = (df['Actual Arrival'] - df['Pickup Date']).dt.days

In [ ]:
# ELIMINACIÓN DE VALORES ATÍPICOS (Días de tránsito no negativo)

# Filtrar para valores lógicos (ej. tiempo de tránsito no negativo)
df = df[df['dias_transito'] >= 0]
print("Valores atípicos eliminados (días de tránsito negativos).")

In [ ]:
df.info()

In [ ]:
df.head()

**ANÁLISIS DE LA VARIABLE OBJETIVO**

In [ ]:
# Histograma de la distribución del tiempo de tránsito

plt.figure(figsize=(10, 6))
sns.histplot(df['dias_transito'], kde=True, bins=30)
plt.title('Distribución de los Días de Tránsito')
plt.xlabel('Días de Tránsito')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Boxplot para identificar valores atípicos (outliers)

plt.figure(figsize=(10, 2))
sns.boxplot(x=df['dias_transito'])
plt.title('Boxplot de Días de Tránsito para identificar outliers')
plt.xlabel('Días de Tránsito')
plt.show()

In [ ]:
# ELIMINACIÓN DE OUTLIERS

# Calculamos el rango intercuartílico (IQR) para 'dias_transito'
Q1 = df['dias_transito'].quantile(0.25)
Q3 = df['dias_transito'].quantile(0.75)
IQR = Q3 - Q1

# Definimos los límites para detectar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtramos el DataFrame, manteniendo solo los valores dentro de los límites
df_sin_outliers = df[(df['dias_transito'] >= lower_bound) & (df['dias_transito'] <= upper_bound)].copy()

print(f"Filas originales: {df.shape[0]}")
print(f"Filas después de eliminar outliers: {df_sin_outliers.shape[0]}")
print(f"Número de outliers eliminados: {df.shape[0] - df_sin_outliers.shape[0]}")

# Usamos el nuevo DataFrame sin outliers para el resto del análisis
df = df_sin_outliers

# Volvemos a visualizar el boxplot para confirmar que los outliers han sido eliminados
plt.figure(figsize=(10, 2))
sns.boxplot(x=df['dias_transito'])
plt.title('Boxplot de Días de Tránsito después de eliminar outliers')
plt.xlabel('Días de Tránsito')
plt.show()

ANÁLISIS DE VARIABLES CATEGÓRICAS

In [ ]:
# Gráfico de barras de los 10 países de origen más comunes (ver si lo dejamos)
plt.figure(figsize=(12, 6))
df['Origin Cntry'].value_counts().nlargest(10).plot(kind='bar')
plt.title('Top 10 Países de Origen')
plt.xlabel('País')
plt.ylabel('Cantidad de Envíos')
plt.xticks(rotation=45, ha='right')
plt.show()

# Gráfico de barras de los niveles de servicio
plt.figure(figsize=(10, 6))
df['Service Level'].value_counts().plot(kind='bar')
plt.title('Frecuencia de Niveles de Servicio')
plt.xlabel('Nivel de Servicio')
plt.ylabel('Cantidad de Envíos')
plt.xticks(rotation=45, ha='right')
plt.show()

**ANÁLISIS DE RELACIONES ENTRE VARIABLES**

In [ ]:
# Gráfico de barras del promedio de días de tránsito por nivel de servicio
# Este gráfico es más útil para comparar promedios que un boxplot.
plt.figure(figsize=(12, 6))
sns.barplot(x='Service Level', y='dias_transito', data=df, errorbar=None)
plt.title('Promedio de Días de Tránsito por Nivel de Servicio')
plt.xlabel('Nivel de Servicio')
plt.ylabel('Promedio de Días de Tránsito')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Gráfico de dispersión con línea de regresión (Scatterplot)
# Este gráfico muestra los puntos individuales y una línea de tendencia para ver la relación.
plt.figure(figsize=(10, 6))
sns.regplot(x='Charge Weight', y='dias_transito', data=df, scatter_kws={'alpha': 0.3})
plt.title('Días de Tránsito vs. Peso de Carga con Línea de Tendencia')
plt.xlabel('Peso de Carga')
plt.ylabel('Días de Tránsito')
plt.show()

ANÁLISIS DE TENDENCIAS Y PATRONES TEMPORALES

In [ ]:
print("\n---")
print("ANÁLISIS DE SERIES DE TIEMPO:")

# Agrupar por fecha para analizar la tendencia de los días de tránsito
# Select only the numerical column 'dias_transito' before resampling and calculating the mean
df_diario = df.set_index('Pickup Date')['dias_transito'].resample('M').mean()

plt.figure(figsize=(15, 6))
plt.plot(df_diario.index, df_diario) # Plot the resampled series directly
plt.title('Tendencia Mensual de Días de Tránsito')
plt.xlabel('Fecha de Recolección')
plt.ylabel('Promedio de Días de Tránsito')
plt.show()

print("\nEDA completado. El dataset está listo para la ingeniería de características y el modelado.")